In [62]:
import sympy as sp
import numpy as np
from IPython.display import display, Math
def sp_skew(v):
   return sp.Matrix([[0, -v[2], v[1]], 
                     [v[2], 0, -v[0]], 
                     [-v[1], v[0], 0]])


def sp_Transformation_matrix(theta , **kwargs):
    dim = 3
    if type(theta) == int or type(theta) == float:
        theta = theta/180*np.pi
    try:
        if kwargs['angle']=='rad':
            theta = theta/np.pi*180
    except KeyError:
        pass

    try:
        if kwargs['dim']!=0:
            dim = kwargs['dim']
    except KeyError:
        pass

    if dim ==3:
        try:
            if kwargs['axis']=='z':
                return sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],[sp.sin(theta),sp.cos(theta),0],[0,0,1]])
            elif kwargs['axis']=='y':
                return sp.Matrix([[sp.cos(theta),0,sp.sin(theta)],[0,1,0],[-sp.sin(theta),0,sp.cos(theta)]])
            elif kwargs['axis']=='x':
                return sp.Matrix([[1,0,0],[0,sp.cos(theta),-sp.sin(theta)],[0,sp.sin(theta),sp.cos(theta)]])
        except:
            return sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],[sp.sin(theta),sp.cos(theta),0],[0,0,1]])
    elif dim==2:
        return sp.Matrix([[sp.cos(theta),-sp.sin(theta)],[sp.sin(theta),sp.cos(theta)]])
    

t = sp.symbols('t')
from sympy.core.numbers import Float

def zero_small(expr, tol=1e-12):
    return expr.replace(
        lambda x: isinstance(x, Float) and abs(x) < tol,
        lambda x: 0
    )
def evaluate(expr, tol=1e-12, digits=4):
    return expr.replace(
        lambda x: isinstance(x, sp.Float),
        lambda x: 0 if abs(x) < tol else round(x, digits)
    )


Kinematics

In [63]:
q = np.array([np.pi/5,np.pi/3])
q_dot = np.array([3, 7])
q_ddot = np.array([4,9])

M = 2
l = 3
I = 1/12 * M * l**2

print("parameters")
display(Math(r'q = ' + sp.latex(q) + r'\quad \dot{q} = ' + sp.latex(q_dot) + r'\quad \ddot{q} = ' + sp.latex(q_ddot) + r'\quad M = ' + str(M) + r'\quad l = ' + str(l) + r'\quad I = ' + str(I)))

A1 = sp_Transformation_matrix(q[0], axis='z')
A2 = A1*sp_Transformation_matrix(q[1], axis='y')

display(Math(r'A1 = ' + sp.latex(A1) + r'\quad A2 = ' + sp.latex(A2.evalf(3))))

s1 = sp.Matrix([0 , l , 0])

r1 = sp.Matrix([0, l/2 , 0  ])
r2 = sp.Matrix([0  , 0 , l/2])

w1p = sp.Matrix([0,0,q_dot[0]])
w2p = sp.Matrix([0,q_dot[1],0])

alpha1p = sp.Matrix([0,0,q_ddot[0]])
alpha2p = sp.Matrix([0,q_ddot[1],0])

w1 = w1p
w2 = w1 + A1 * w2p

alpha1 = alpha1p
alpha2 = alpha1 + sp_skew(w1) * A1 * w2p + A1 * alpha2p


a1 = sp_skew(alpha1) * A1 * r1 + sp_skew(w1) * sp_skew(w1) * A1 * r1
a2 = (sp_skew(alpha1) * A1 * s1 + sp_skew(w1) * sp_skew(w1) * A1 * s1 + 
      sp_skew(alpha2) * A2 * r2 + sp_skew(w2) * sp_skew(w2) * A2 * r2)
display(Math(r'Symbolic \; a_1: \tilde{a}_1 s_1 + \tilde{w_1} \tilde{w_1} s_1 = a_1'))
display(Math(r'Symbolic \; a_2: \tilde{a}_1 s_1 + \tilde{w_1} \tilde{w_1} s_1 + \tilde{a_2} r_2 + \tilde{w_2} \tilde{w_2} r_2 = a_2'))

print("kinematics")

display(Math(r'w1 = ' + sp.latex(w1) + r'\quad w2 = ' + sp.latex(w2) + r'\quad \alpha1 = ' + sp.latex(alpha1) + r'\quad \alpha2 = ' + sp.latex(alpha2)))
display(Math(r'a1 = ' + sp.latex(a1) + r'\quad a2 = ' + sp.latex(a2.evalf(3)) ))

parameters


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

kinematics


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Dynamics

In [64]:
# Definitions
M1 = sp.Matrix([[M, 0, 0], [0, M, 0], [0, 0, M]])
M2 = sp.Matrix([[M, 0, 0], [0, M, 0], [0, 0, M]])

I1 = A1 * sp.Matrix([[I, 0, 0], [0, 0, 0], [0, 0, I]]) * A1.inv()
I2 = A2 * sp.Matrix([[I, 0, 0], [0, I, 0], [0, 0, 0]]) * A2.inv()

display(Math(r'M1 = ' + sp.latex(M1) +
             r'\quad M2 = ' + sp.latex(M2) +
             r'\quad I1 = ' + sp.latex(evaluate(A1 * I1)) +
             r'\quad I2 = ' + sp.latex(evaluate(I2))))

g = sp.Matrix([0, 0, -9.81])

Force2 =  M2 * a2 - M2 * g
Torque2 = I2 * alpha2 + sp_skew(w2) * I2 * w2 - sp_skew(A2 * (-r2)) * Force2
Force1 = M1 * a1 + Force2 - M1 * g
Torque1 = I1 * alpha1 + sp_skew(w1) * I1 * w1 - sp_skew(A1 * r1) * (-Force2) - sp_skew(A1 * (-r1)) * (Force1) + Torque2

display(Math(r'Symbolic \; R_2: \quad \quad -m \cdot a_2 + m \cdot g = F_2'))
display(Math(r'Symbolic \; M_2: \quad \quad \quad I \cdot \alpha_2 + \omega_2 \times I_2 \cdot \omega_2 + (-r_2) \times (-R_2) = T_2'))
display(Math(r'Symbolic \; R_1: \quad \quad -m \cdot a_1 + F_2 + m \cdot g = F_1'))
display(Math(r'Symbolic \; M_1: \quad \quad I \cdot \alpha_1 + \omega_1 \times I_1 \cdot \omega_1 - r_1 \times F_2 + r_1 \times F_1 + T_2 = T_1'))


print("Equations of motion (symbolic)")
display(Math(r'F = m \cdot a \quad T = I \cdot \alpha + \omega \times I \cdot \omega + r \times m \cdot a'))

print("forces and torques")
display(Math(
    r'F1 = ' + sp.latex(evaluate(A1.inv()*Force1)) +
    r'\quad M1 = ' + sp.latex(evaluate(A1.inv()*Torque1, digits=15)) +
    r'\quad F2 = ' + sp.latex(evaluate(A2.inv()*Force2)) +
    r'\quad M2 = ' + sp.latex(evaluate(A2.inv()*Torque2, digits=15))
))

print("Joint torques")
display(Math(
    r'\tau_1 = ' + sp.latex(evaluate(A1.inv()*Torque1)[2]) +
    r'\quad \tau_2 = ' + sp.latex(evaluate(A2.inv()*Torque2)[1])
))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

forces and torques


<IPython.core.display.Math object>

Joint torques


<IPython.core.display.Math object>

## C++ implementation

In [65]:
import sys
import os
import inspect
import numpy as np
import matplotlib.pyplot as plt

current_dir = os.path.dirname(
    os.path.dirname(
        os.path.abspath(inspect.getsourcefile(lambda:0))
    )
)
module_path = os.path.join(
    current_dir, 
    "robot_dynamics_module", 
    "build", 
    "Release"
)
sys.path.append(module_path)

In [66]:
# Import the compiled C++ torque calculator
import freecad_robotics.compute_torque as compute_torque


DH_params = [
    [0.0, 0.0, 0.0, np.pi/2],
    [0.0, 0.0, 0.0, 0.0]
]

# Masses
M_arr = np.array([M, M])   # m1 = m2 = 2 kg

# Inertia (diagonal) for each link
I_val = (1/12) * M * l**2
InertiaMatrices = [
    np.array([[I_val, 0,     0],
              [0,     I_val, 0],
              [0,     0,     0]]),
    np.array([[I_val, 0,     0],
              [0,     0,     0],
              [0,     0,     I_val]])
]

# Centers of mass in each link frame
CenterOfMass = np.array([
    [0,    0,   l/2],
    [0,    l/2, l  ]
])


# Compute torques
tau_cpp = compute_torque.computeJointTorques(
    q, q_dot, q_ddot,
    M_arr, InertiaMatrices, CenterOfMass,
    np.array(DH_params)
)


D_mat, C_mat, g_arr = compute_torque.getMatrices(
    q, q_dot,
    M_arr, InertiaMatrices, CenterOfMass,
    DH_params
)

# Print the results
display(Math(
    r'D(q) = ' + sp.latex(evaluate(sp.Matrix(D_mat))) +
    r'\quad C(\dot{q},q) = ' + sp.latex(evaluate(sp.Matrix(C_mat))) +
    r'\quad g(q) = ' + sp.latex(evaluate(sp.Matrix(g_arr)))
))

def print_torques(label, tau):
    print(f"{label} Torque Output:")
    for i, t in enumerate(tau):
        print(f"  Joint {i+1}: {t:.4f}")
    print()

print("Planar Robot Experiment (Static Configuration)")
print("================================================")
display(Math(
    r'q = ' + sp.latex(evaluate(sp.Matrix(q))) +
    r'\quad \dot{q} = ' + sp.latex(sp.Matrix(q_dot)) +
    r'\quad \ddot{q} = ' + sp.latex(sp.Matrix(q_ddot))))
print()

print_torques("C++ Module", tau_cpp)

<IPython.core.display.Math object>

Planar Robot Experiment (Static Configuration)


<IPython.core.display.Math object>


C++ Module Torque Output:
  Joint 1: 564.5364
  Joint 2: -12.8698



In [67]:
# Robotics toolbox only works with numpy < 2.0.0. Create an environment with numpy < 2.0.0 to run this:


def compute_torque_roboticstoolbox(
    q, q_dot, q_ddot, M, InertiaMatrices, CenterOfMass, DH_params
):
    from roboticstoolbox import DHRobot, RevoluteDH

    n_dof = len(DH_params)
    links = []
    for i in range(n_dof):
        θ_off, d, a, α = DH_params[i]
        link = RevoluteDH(d=d, a=a, alpha=α)
        link.m = M[i]
        link.r = CenterOfMass[i]
        link.I = InertiaMatrices[i]
        links.append(link)

    robot = DHRobot(links)
    gravity = np.array([0, 0, -9.81])
    tau = robot.rne(q, q_dot, q_ddot, gravity)
    return tau


try:
    tau_rtb = compute_torque_roboticstoolbox(
        q, q_dot, q_ddot,
        M_arr, InertiaMatrices, CenterOfMass,
        DH_params
    )
except:
    print("Robotics Toolbox is not available. Skipping this part.")
    print("Make sure you have the toolbox installed and numpy < 2.0.0.")
    tau_rtb = np.zeros(len(q))


print_torques("Robotics Toolbox", tau_rtb)

Robotics Toolbox is not available. Skipping this part.
Make sure you have the toolbox installed and numpy < 2.0.0.
Robotics Toolbox Torque Output:
  Joint 1: 0.0000
  Joint 2: 0.0000

